Module importieren

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
#import pd as pd
import seaborn as sns
from datetime import datetime
from fitter import Fitter
import numpy as np
from scipy import stats
import statsmodels.api as sm
from tabulate import tabulate

Daten laden und säubern

In [2]:
def open_csv_to_df(directory):
    df = pd.read_csv(directory)
    df = df.iloc[0:,:4]
    df.columns = ["Date", "Time", "Mass_(kg)", "Velocity_(m/s)"]
    df.insert(4, "Energy_(kJ)", "")
    df.insert(5, "Date_and_Time", "")
    df.insert(6, "Time_delta_(h)", "")
    for i in range(len(df)):
        df.iloc[i,4] = 0.5 * float(df.iloc[i,2]) * float(df.iloc[i,3]) ** 2 / 1000 #calculation of Energy_(kJ)
    df = df.sort_values(['Date', 'Time']).reset_index(drop=True)
    df = df.dropna()

    for i in range(len(df)):
        date_raw = df.iloc[i,:].Date +" " +df.iloc[i,:].Time
        date1 = datetime.strptime(date_raw, '%Y-%m-%d %H:%M')
        df.iloc[i,5] = date1

    for i in range(len(df)-1):
        date1 = df.iloc[i,5]
        date2 = df.iloc[i+1,5]
        time_delta = date2 - date1
        time_delta = (time_delta.days*24) + (time_delta.seconds//3600)
        df.iloc[i+1,6] = time_delta

    #Fitter kann keine NAs handhaben aus diesem Grund wird erste Beobachtung mit dem Median der Zeitabstände ersetzt.
    df.iloc[0,6] = 0
    df['Time_delta_(h)'] = df['Time_delta_(h)'].astype('int')
    df.iloc[0,6] = df["Time_delta_(h)"].median()

    return df #this is the table with all stones combined

In [3]:
df1 = open_csv_to_df("./out_1.csv")
df2 = open_csv_to_df("./out_2.csv")

Funktion für Fitting von Verteilungen

In [4]:
def fit_distribution(data):
    count = 0
    fitted_mass = Fitter(data)
    fitted_mass.fit()
    ks_summary = fitted_mass.summary()
    distributions = ks_summary.index.values
    print(ks_summary.iloc[:,-2:])
    fig, ax = plt.subplots(1,5,figsize=(25, 5))
    table = [['Distribution', 'Parameters']]

    for i in distributions:
        param = fitted_mass.fitted_param[i]
        table.append([i, param])
        dist_eval = eval('stats.' + i + '.rvs(*param, size = 1000)')
        sm.qqplot_2samples(data, dist_eval, xlabel = i.capitalize() + ' Distribution', ylabel = 'Sample Distribution', line = '45' ,ax = ax[count])
        count += 1
    plt.show()
    print(tabulate(table, headers = 'firstrow'))

Zone 1

In [5]:
#fit_distribution(df1['Mass_(kg)'])

In [6]:
#fit_distribution(df1["Velocity_(m/s)"])

In [7]:
#fit_distribution(df1["Time_delta_(h)"])

Zone 2

In [8]:
#fit_distribution(df2['Mass_(kg)'])

In [9]:
#fit_distribution(df2["Velocity_(m/s)"])

In [10]:
#fit_distribution(df2["Time_delta_(h)"])

Monte Carlo

In [11]:
sim_count = 3_000_000


Zone 1

In [12]:
# Masse
param = stats.gamma.fit(df1['Mass_(kg)'])
zone1_mass = stats.gamma.rvs(*param, size = sim_count)

# Geschwindigkeit
param = stats.norm.fit(df1["Velocity_(m/s)"])
zone1_velocity = stats.norm.rvs(*param, size = sim_count)

# Zeitabstand
param = stats.expon.fit(df1["Time_delta_(h)"])
zone1_time_delta = stats.expon.rvs(*param, size = sim_count)

Zone 2

In [13]:
# Masse
param = stats.gamma.fit(df2['Mass_(kg)'])
zone2_mass = stats.gamma.rvs(*param, size = sim_count)

# Geschwindigkeit
param = stats.norm.fit(df2["Velocity_(m/s)"])
zone2_velocity = stats.norm.rvs(*param, size = sim_count)

# Zeitabstand
param = stats.expon.fit(df2["Time_delta_(h)"])
zone2_time_delta = stats.expon.rvs(*param, size = sim_count)

In [14]:
df1_sim = pd.DataFrame()
df1_sim.insert(0, "Mass_(kg)", zone1_mass)
df1_sim.insert(1, "Velocity_(m/s)", zone1_velocity)
df1_sim.insert(2, "Energy_(kJ)", "")
df1_sim.insert(3, "Time_delta_(h)", zone1_time_delta)


df1_sim["Time_delta_(h)"] = zone1_time_delta.round(4)
df1_sim["Time_delta_(h)"][0] = 0

df1_sim = df1_sim.dropna()
df1_sim["Energy_(kJ)"] = (0.5 * df1_sim["Mass_(kg)"] * (df1_sim["Velocity_(m/s)"] ** 2)) / 1000
df1_sim

C:\Users\logan\AppData\Local\Temp\ipykernel_16508\3105050015.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sim["Time_delta_(h)"][0] = 0


,Mass_(kg),Velocity_(m/s),Energy_(kJ),Time_delta_(h)
0,1865.569662,10.780719,108.411901,0.0000
1,488.669032,9.984007,24.355363,42.8974
2,559.954952,11.187662,35.043041,4.0537
3,947.847815,7.981338,30.189785,126.3390
4,622.346400,11.186514,38.939620,6.7256
...,...,...,...,...
2999995,567.104539,8.264953,19.369298,49.2674
2999996,142.077294,5.894859,2.468548,182.1120
2999997,621.289294,10.151269,32.011388,32.4555
2999998,83.142022,6.499546,1.756130,15.3397


In [15]:
df2_sim = pd.DataFrame()
df2_sim.insert(0, "Mass_(kg)", zone2_mass)
df2_sim.insert(1, "Velocity_(m/s)", zone2_velocity)
df2_sim.insert(2, "Energy_(kJ)", "")
df2_sim.insert(3, "Time_delta_(h)", zone2_time_delta)


df2_sim["Time_delta_(h)"] = zone2_time_delta.round(4)
df2_sim["Time_delta_(h)"][0] = 0

df2_sim = df2_sim.dropna()
df2_sim["Energy_(kJ)"] = (0.5 * df2_sim["Mass_(kg)"] * (df2_sim["Velocity_(m/s)"] ** 2)) / 1000
df2_sim

C:\Users\logan\AppData\Local\Temp\ipykernel_16508\2961706192.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_sim["Time_delta_(h)"][0] = 0


,Mass_(kg),Velocity_(m/s),Energy_(kJ),Time_delta_(h)
0,104.845664,42.212107,93.410253,0.0000
1,58.448791,41.056823,49.262473,13.0124
2,71.965716,29.676396,31.689689,90.7431
3,64.280605,37.419224,45.002799,8.7980
4,4.396340,51.008340,5.719310,41.3349
...,...,...,...,...
2999995,16.121771,40.536815,13.245916,62.0449
2999996,6.245906,35.238045,3.877833,99.0385
2999997,12.517941,35.836983,8.038329,40.4893
2999998,231.317507,31.069513,111.647088,23.7023


In [16]:
df_sim = pd.concat([df1_sim,df2_sim])
df_sim = df_sim.sample(frac=1,ignore_index=True)

In [17]:
def netztest(status_netz,gewicht_im_netz,aufprallenergie):

    if status_netz == True:
        return True

    elif gewicht_im_netz > 2000:
        if aufprallenergie > 500:
            return True
        else:
            return False

    elif gewicht_im_netz <= 2000:
        if aufprallenergie > 1000:
            return True
        else:
            return False
    else:
        return False

In [18]:
df_sim

,Mass_(kg),Velocity_(m/s),Energy_(kJ),Time_delta_(h)
0,3.992336,36.964388,2.727496,14.7928
1,82.068180,9.151233,3.436403,3.8886
2,139.034487,41.110814,117.491023,76.7989
3,464.207896,10.908690,27.620257,27.4309
4,875.417822,11.489703,57.783384,20.8659
...,...,...,...,...
5999995,121.234526,6.547680,2.598790,64.7772
5999996,673.657348,12.013370,48.611471,0.6089
5999997,2075.254265,9.133083,86.551802,11.4246
5999998,489.130582,5.898376,8.508631,9.6105


In [19]:
max_delta = 24
current_delta = 0
stunden_total = 0
masse_im_netz = 0
status_netz = False
incident_steine = 0
steine_im_netz = 0
length = len(df_sim)

for i in range(int(length)):
    iterated_delta = df_sim["Time_delta_(h)"][i]
    iterated_energy = df_sim["Energy_(kJ)"][i]

    stunden_total += iterated_delta

    if (iterated_delta > max_delta):
        if status_netz:
            incident_steine += steine_im_netz
        steine_im_netz = 1
        status_netz = netztest(False,0,iterated_energy)
        masse_im_netz = df_sim["Mass_(kg)"][i]

        iterated_delta = iterated_delta % max_delta


        if iterated_delta > (max_delta - current_delta):

            current_delta = abs((max_delta - current_delta)-iterated_delta)


        elif iterated_delta <= (max_delta - current_delta):

            current_delta = current_delta + iterated_delta

    elif iterated_delta <= max_delta:

        if iterated_delta > (max_delta - current_delta):
            if status_netz:
                incident_steine += steine_im_netz
            steine_im_netz = 1
            status_netz = netztest(False,0,iterated_energy)
            masse_im_netz = df_sim["Mass_(kg)"][i]
            current_delta = abs((max_delta - current_delta)-iterated_delta)

        elif iterated_delta <= (max_delta - current_delta):
            steine_im_netz += 1
            status_netz = netztest(status_netz,masse_im_netz,iterated_energy)
            masse_im_netz += df_sim["Mass_(kg)"][i]
            current_delta = current_delta + iterated_delta



print("Anzahl simulierte Steine: ",length)
print("Anzahl incident Steine: ",incident_steine)
print(incident_steine/length*100)

Anzahl simulierte Steine:  6000000
Anzahl incident Steine:  373
0.006216666666666667


In [20]:
anzahl_sim_jahre = stunden_total / 8760
steine_pro_jahr = (length / anzahl_sim_jahre)
incident_prob_pro_stein = incident_steine/length
incident_prob_pro_jahr = incident_prob_pro_stein * steine_pro_jahr
print(incident_prob_pro_jahr)



0.011317910187633637


In [31]:
def danger_zone(v_km_h,laenge_auto,autos_pro_tag):
    seconds_per_year = 86400 * 365
    v_m_s = (v_km_h / 3.6)
    danger_time_pro_auto = (laenge_auto / v_m_s)
    autos_pro_jahr = autos_pro_tag * 365
    print(danger_time_pro_auto)
    danger_time_total = danger_time_pro_auto * autos_pro_jahr
    print(danger_time_total)
    danger_percentage = (danger_time_total / seconds_per_year)
    print(danger_percentage)

    return danger_percentage

In [32]:
danger_strecke = danger_zone(60,5,1200)

0.3
131400.0
0.004166666666666667


In [33]:
print(incident_prob_pro_jahr)
print(danger_strecke)
print('{:f}'.format(incident_prob_pro_jahr * danger_strecke ))

0.011317910187633637
0.004166666666666667
0.000047
